# 추천 시스템 (Recommender Systems)

* 추천 시스템은 크게 두가지로 구분 가능
  * 컨텐츠 기반 필터링 (content-based filtering)
  * 협업 필터링 (collaborative filtering)
* 두가지를 조합한 hybrid 방식도 가능
* 컨텐츠 기반 필터링은 지금까지 사용자의 이전 행동과 명시적 피드백을 통해 사용자가 좋아하는 것과 유사한 항목을 추천
* 협업 필터링은 사용자와 항목간의 유사성을 동시에 사용해 추천

## Surprise

* 추천 시스템 개발을 위한 라이브러리
* 다양한 모델과 데이터 제공
* scikit-learn과 유사한 사용 방법

In [1]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8MB 349kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1670932 sha256=47f347831434efab51565d97878c403f6d5463d6ad407d30e1e5f7c594986468
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


간단한 surprise 실습

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

In [3]:
#영화 데이터 10개만
data = Dataset.load_builtin('ml-100k', prompt= False)
data.raw_ratings[:10]

#196번째 유저,242의 영화를 3.0점 줌 그 아이디는 881250949

Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013')]

In [4]:
model = SVD()

In [8]:
# 싱글로 벨류 디컴포지션을 사용해볼게
cross_validate(model,data, measures = ['rmse', 'mae'], cv = 5,verbose =True) # cross validation =5 개

# fold가 5개 -> cv 가 5개이니까

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9344  0.9346  0.9351  0.9443  0.9309  0.9359  0.0045  
MAE (testset)     0.7376  0.7378  0.7360  0.7434  0.7351  0.7380  0.0029  
Fit time          4.97    5.04    5.03    5.04    5.05    5.02    0.03    
Test time         0.16    0.27    0.14    0.22    0.13    0.18    0.05    


{'fit_time': (4.965756893157959,
  5.036713361740112,
  5.02909779548645,
  5.036122560501099,
  5.051944017410278),
 'test_mae': array([0.73757663, 0.73781866, 0.73603319, 0.74336826, 0.73505216]),
 'test_rmse': array([0.93441732, 0.93455173, 0.93514883, 0.94429485, 0.93093382]),
 'test_time': (0.16006731986999512,
  0.27309155464172363,
  0.13614487648010254,
  0.2166881561279297,
  0.13396072387695312)}

## 컨텐츠 기반 필터링 (Content-based Filtering)

* 컨텐츠 기반 필터링은 이전의 행동과 명시적 피드백을 통해 좋아하는 것과 유사한 항목을 추천
  * ex) 내가 지금 까지 시청한 영화 목록과 다른 사용자의 시청 목록을 비교해 나와 비슷한 취향의 사용자가 시청한 영화를 추천
* 유사도를 기반으로 추천
* 컨텐츠 기반 필터링은 다음과 같은 장단점이 있다.
  * 장점
    * 많은 수의 사용자를 대상으로 쉽게 확장 가능
    * 사용자가 관심을 갖지 않던 상품 추천 가능
  * 단점
    * 입력 특성을 직접 설계해야 하기 때문에 많은 도메인 지식이 필요
    * 사용자의 기존 관심사항을 기반으로만 추천 가능

In [9]:
import numpy as np
from surprise import Dataset

* 이진 벡터의 내적을 통해 다른 사용자들과의 유사도 구하기
* 나와 가장 높은 유사도를 가진 사용자의 시청 목록을 추천

In [11]:
data = Dataset.load_builtin('ml-100k', prompt = False)
raw_data = np.array(data.raw_ratings, dtype = int)


In [13]:
# 0부터 시작할수 있게 조정하고
raw_data[:,0] -= 1
raw_data[:,1] -= 1

In [14]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users + 1 , n_movies + 1)
shape

(942, 1681)

In [21]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = 1

adj_matrix

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 1]])

In [23]:
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        similarity = np.dot(my_vector,user_vector)
        if similarity > best_match:
            best_match = similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))

best match : 54, best match id : 653


In [24]:
recommend_list = []
for i , log in enumerate(zip(my_vector,best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 >0.:
        recommend_list.append(i)

print(recommend_list)

[0, 2, 3, 4, 5, 6, 7, 9, 10, 13, 16, 18, 19, 20, 21, 22, 24, 25, 26, 28, 29, 30, 34, 36, 40, 41, 42, 43, 44, 45, 46, 47, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 67, 68, 74, 75, 77, 79, 80, 84, 85, 86, 87, 90, 91, 94, 95, 96, 111, 114, 115, 116, 119, 120, 122, 123, 124, 126, 127, 129, 130, 131, 132, 133, 135, 141, 142, 147, 148, 150, 151, 153, 154, 155, 157, 158, 159, 160, 162, 163, 164, 165, 168, 169, 170, 172, 173, 174, 176, 177, 179, 180, 181, 183, 184, 185, 186, 188, 189, 190, 191, 193, 194, 195, 196, 198, 200, 201, 202, 203, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 222, 224, 226, 231, 232, 234, 236, 237, 238, 244, 246, 247, 248, 250, 254, 260, 263, 266, 268, 269, 315, 316, 317, 318, 319, 322, 323, 324, 325, 326, 328, 330, 331, 335, 338, 342, 343, 344, 345, 352, 354, 355, 357, 361, 365, 369, 370, 373, 376, 379, 380, 383, 385, 389, 391, 394, 400, 401, 403, 408, 409, 415, 421, 423, 425, 426, 431, 433, 441, 445, 446, 447, 449, 452,

* 유클리드 거리를 사용해 추천
$$euclidean = \sqrt{\sum_{d=1}^{D}(A_i - B_i)^2}$$
* 거리가 가까울 수록(값이 작을 수록) 나와 유사한 사용자

In [25]:
my_id, my_vector = 0, adj_matrix[0]# 아이다가 0이라 가정하고
best_match, best_match_id, best_match_vector = 9999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))

best match : 6.855654600401044, best match id : 699


In [26]:
recommend_list = []
for i , log in enumerate(zip(my_vector,best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 >0.:
        recommend_list.append(i)

print(recommend_list)

[122, 324, 326, 331, 342, 687, 688, 748, 749]


* 코사인 유사도를 사용해 추천

\begin{equation}
cos \theta = \frac{A \cdot B}{||A|| \times ||B||}
\end{equation}
* 두 벡터가 이루고 있는 각을 계산

In [27]:
def compute_cos_similarity(v1,v2):
    norm1 = np.sqrt(np.sum(np.square(v1)))
    norm2 = np.sqrt(np.sum(np.square(v2)))
    dot = np.dot(v1,v2)# 분자역할

    return dot / (norm1 * norm2) # 분모 역할

In [30]:
my_id, my_vector = 0, adj_matrix[0]        # 아이다가 0이라 가정하고
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))

best match : 0.5305892259424583, best match id : 699


In [31]:
recommend_list = []
for i , log in enumerate(zip(my_vector,best_match_vector)):
    log1, log2 = log
    if log1 < 1. and log2 >0.:
        recommend_list.append(i)

print(recommend_list)

[122, 324, 326, 331, 342, 687, 688, 748, 749]


기존 방법에 명시적 피드백(사용자가 평가한 영화 점수)을 추가해 실험

In [35]:
adj_matrix = np.ndarray(shape, dtype = int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
adj_matrix

array([[0, 0, 0, ..., 0, 0, 4],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 5],
       [0, 0, 0, ..., 0, 0, 0],
       [5, 4, 3, ..., 0, 0, 5]])

In [36]:
#유클리디안으로
my_id, my_vector = 0, adj_matrix[0]
best_match, best_match_id, best_match_vector = 9999,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        euclidean_dist = np.sqrt(np.sum(np.square(my_vector - user_vector)))
        if euclidean_dist < best_match:
            best_match = euclidean_dist
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))

best match : 25.632011235952593, best match id : 699


In [37]:
#cos로 돌려볼게
my_id, my_vector = 0, adj_matrix[0]        # 아이다가 0이라 가정하고
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(adj_matrix):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))

# 같게 나왓지만 원래는 다르게 나와야해

best match : 0.5806357207498616, best match id : 699


## 협업 필터링(Collaborative Filtering)

* 사용자와 항목의 유사성을 동시에 고려해 추천
* 기존에 내 관심사가 아닌 항목이라도 추천 가능
* 자동으로 임베딩 학습 가능
* 협업 필터링은 다음과 같은 장단점을 갖고 있다.
  * 장점
    * 자동으로 임베딩을 학습하기 때문에 도메인 지식이 필요 없다.
    * 기존의 관심사가 아니더라도 추천 가능
  * 단점
    * 학습 과정에 나오지 않은 항목은 임베딩을 만들 수 없음
    * 추가 특성을 사용하기 어려움

In [41]:
from surprise import KNNBasic, SVD, SVDpp, NMF
from surprise import Dataset
from surprise.model_selection import cross_validate

In [42]:
data = Dataset.load_builtin('ml-100k', prompt = False)

* KNN을 사용한 협업 필터링

In [44]:
model = KNNBasic()
cross_validate(model, data, measures = ['rmse', 'mae'], cv= 5, n_jobs=4, verbose = True)

Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9775  0.9788  0.9767  0.9792  0.9866  0.9797  0.0035  
MAE (testset)     0.7731  0.7730  0.7712  0.7727  0.7790  0.7738  0.0027  
Fit time          0.41    0.61    0.91    0.97    0.52    0.68    0.22    
Test time         7.91    9.70    9.82    7.50    5.44    8.07    1.61    


{'fit_time': (0.4068162441253662,
  0.6098010540008545,
  0.9122862815856934,
  0.9744484424591064,
  0.5176296234130859),
 'test_mae': array([0.77311353, 0.77298086, 0.77122568, 0.77273164, 0.77900273]),
 'test_rmse': array([0.97745198, 0.97881161, 0.9766645 , 0.97915035, 0.9865944 ]),
 'test_time': (7.9123852252960205,
  9.697359323501587,
  9.818226099014282,
  7.496584177017212,
  5.443845748901367)}

* SVD를 사용한 협업 필터링

In [45]:
model = SVD()
cross_validate(model, data, measures = ['rmse', 'mae'], cv= 5, n_jobs=4, verbose = True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9297  0.9326  0.9367  0.9403  0.9339  0.9346  0.0036  
MAE (testset)     0.7326  0.7364  0.7365  0.7415  0.7338  0.7362  0.0031  
Fit time          16.21   19.42   19.26   17.08   11.99   16.79   2.70    
Test time         0.57    0.54    0.41    0.26    0.16    0.39    0.16    


{'fit_time': (16.213504314422607,
  19.421885013580322,
  19.255165576934814,
  17.08173441886902,
  11.991374015808105),
 'test_mae': array([0.73259018, 0.73637639, 0.73651277, 0.74148588, 0.73382573]),
 'test_rmse': array([0.92974121, 0.93256714, 0.93666847, 0.94034501, 0.93388478]),
 'test_time': (0.5683422088623047,
  0.5382571220397949,
  0.40862226486206055,
  0.2553896903991699,
  0.1561882495880127)}

* NMF를 사용한 협업 필터링

In [46]:
model = NMF()
cross_validate(model, data, measures = ['rmse', 'mae'], cv= 5, n_jobs=4, verbose = True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9694  0.9604  0.9585  0.9674  0.9673  0.9646  0.0043  
MAE (testset)     0.7585  0.7541  0.7532  0.7632  0.7610  0.7580  0.0038  
Fit time          13.79   17.01   16.92   14.59   10.57   14.58   2.37    
Test time         0.58    0.44    0.32    0.21    0.13    0.34    0.16    


{'fit_time': (13.790802717208862,
  17.01211404800415,
  16.91950488090515,
  14.586909055709839,
  10.570446252822876),
 'test_mae': array([0.75851272, 0.75408525, 0.75324533, 0.76317226, 0.76102158]),
 'test_rmse': array([0.9694137 , 0.96042034, 0.95854142, 0.96739843, 0.96733878]),
 'test_time': (0.5758578777313232,
  0.4435889720916748,
  0.31651735305786133,
  0.2148442268371582,
  0.13020730018615723)}

* SVD++를 사용한 협업 필터링

In [47]:
# 너무 오래걸림 사실상 사용하기 어려운

model = SVDpp()
cross_validate(model, data, measures = ['rmse', 'mae'], cv= 5, n_jobs=4, verbose = True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9254  0.9131  0.9278  0.9105  0.9175  0.9189  0.0068  
MAE (testset)     0.7236  0.7130  0.7276  0.7152  0.7214  0.7202  0.0054  
Fit time          657.13  662.23  658.94  664.76  169.56  562.52  196.50  
Test time         12.09   12.01   12.21   6.55    3.10    9.19    3.73    


{'fit_time': (657.1291942596436,
  662.232207775116,
  658.9355690479279,
  664.7564654350281,
  169.5636432170868),
 'test_mae': array([0.72359603, 0.71304114, 0.72759631, 0.71521399, 0.7213561 ]),
 'test_rmse': array([0.92543382, 0.91311402, 0.92783046, 0.91049074, 0.91746388]),
 'test_time': (12.092623472213745,
  12.014078378677368,
  12.211187601089478,
  6.546313047409058,
  3.1020069122314453)}

## 하이브리드(Hybrid)

* 컨텐츠 기반 필터링과 협업 필터링을 조합한 방식
* 많은 하이브리드 방식이 존재
* 실습에서는 협업 필터링으로 임베딩을 학습하고 컨텐츠 기반 필터링으로 유사도 기반 추천을 수행하는 추천 엔진 개발

In [49]:
import numpy as np
from sklearn.decomposition import randomized_svd, non_negative_factorization 
from surprise import Dataset


In [78]:
data = Dataset.load_builtin('ml-100k', prompt= False)
raw_data = np.array(data.raw_ratings, dtype = int)
raw_data[:,0] -=1 
raw_data[:,1] -=1

In [79]:
n_users = np.max(raw_data[:,0])
n_movies = np.max(raw_data[:,1])
shape = (n_users +1, n_movies + 1)
shape

(943, 1682)

In [80]:
adj_matrix = np.ndarray(shape,dtype = int)
for user_id, movie_id, rating, time in raw_data:
    adj_matrix[user_id][movie_id] = rating
    

In [81]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [82]:
U, S, V = randomized_svd(adj_matrix, n_components=2)# 2차원으로
S = np.diag(S)


In [83]:
print(U.shape)
print(S.shape) # 잠재요인을 찾을수 있는 특이값 vector가 되어줌
print(V.shape)

(943, 2)
(2, 2)
(2, 1682)


In [84]:
np.matmul(np.matmul(U,S),V)

array([[ 3.91732663e+00,  1.47276644e+00,  7.98261988e-01, ...,
         6.24907189e-04,  1.41100852e-02,  1.36545878e-02],
       [ 1.85777226e+00,  3.96191175e-01,  5.05705740e-01, ...,
         5.38862978e-03,  1.77237914e-03,  5.26968095e-04],
       [ 8.94989517e-01,  1.71578497e-01,  2.51738682e-01, ...,
         2.92094923e-03,  5.39937171e-04, -1.25733753e-04],
       ...,
       [ 9.92051955e-01,  2.10814957e-01,  2.70363365e-01, ...,
         2.89019297e-03,  9.34221962e-04,  2.66612193e-04],
       [ 1.30425401e+00,  5.27669941e-01,  2.50080165e-01, ...,
        -4.20677765e-04,  5.30525683e-03,  5.28069948e-03],
       [ 2.82999397e+00,  9.70812247e-01,  6.15871694e-01, ...,
         2.02091492e-03,  8.67740813e-03,  8.03107892e-03]])

* 사용자 기반 추천
* 나와 비슷한 취향을 가진 다른 사용자의 행동을 추천
* 사용자 특징 벡터의 유사도 사용

In [85]:
my_id, my_vector = 0, U[0] # u니까 user에 대해서
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))

# 하이브리드로 -> SVD 돌리고 COS로 돌리니까 99% 나왓네?

best match : 0.9999942295956324, best match id : 235


In [86]:
recommend_list = []
for i , log in enumerate(zip(adj_matrix[my_id],adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 >0.:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


* 항목 기반 추천
* 내가 본 항목과 비슷한 항목을 추천
* 항목 특징 벡터의 유사도 사용

In [87]:
my_id, my_vector = 0, V.T[0]  # 항목 V, 곱으로 디컴포션 해줬으니까 T 해주자
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))


best match : 0.9999999951364141, best match id : 1287


In [88]:
recommend_list = []
for i , user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)

print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 비음수 행렬 분해를 사용한 하이브리드 추천

In [89]:
adj_matrix

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [5, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 5, 0, ..., 0, 0, 0]])

In [90]:
A,B , iter = non_negative_factorization(adj_matrix , n_components=2)

/usr/local/lib/python3.6/dist-packages/sklearn/decomposition/_nmf.py:1035: FutureWarning: The default value of init will change from random to None in 0.23 to make it consistent with decomposition.NMF.
  "with decomposition.NMF.", FutureWarning)


In [91]:
np.matmul(A,B)

array([[3.71104010e+00, 1.48480406e+00, 7.39556361e-01, ...,
        3.64528799e-03, 1.45531081e-02, 1.44128911e-02],
       [2.11742256e+00, 2.36684061e-01, 5.51606409e-01, ...,
        4.76766262e-03, 2.39224998e-05, 0.00000000e+00],
       [9.85451191e-01, 1.10153067e-01, 2.56718335e-01, ...,
        2.21887633e-03, 1.11335623e-05, 0.00000000e+00],
       ...,
       [1.04460771e+00, 1.16765543e-01, 2.72129105e-01, ...,
        2.35207521e-03, 1.18019086e-05, 0.00000000e+00],
       [1.45794397e+00, 5.42097653e-01, 2.99302325e-01, ...,
        1.61363785e-03, 5.15823673e-03, 5.10693782e-03],
       [2.44654659e+00, 9.41446354e-01, 4.95508656e-01, ...,
        2.56797603e-03, 9.08670615e-03, 8.99770468e-03]])

사용자 기반

In [92]:
my_id, my_vector = 0, U[0]  # 항목 V, 곱으로 디컴포션 해줬으니까 T 해주자
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(U):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))


best match : 0.9999942295956324, best match id : 235


In [93]:
recommend_list = []
for i , log in enumerate(zip(adj_matrix[my_id],adj_matrix[best_match_id])):
    log1, log2 = log
    if log1 < 1. and log2 >0.:
        recommend_list.append(i)

print(recommend_list)

[272, 273, 274, 281, 285, 288, 293, 297, 303, 306, 312, 317, 327, 332, 369, 410, 418, 419, 422, 426, 428, 431, 434, 442, 461, 475, 477, 482, 495, 503, 504, 505, 506, 509, 519, 520, 522, 525, 531, 545, 548, 590, 594, 595, 613, 631, 654, 658, 660, 672, 684, 685, 691, 695, 698, 704, 716, 728, 734, 749, 755, 863, 865, 933, 1012, 1038, 1101, 1327, 1400]


항목 기반

In [95]:
my_id, my_vector = 0, V.T[0]  # 기준 항목이 0이라 가정 ㅋ
best_match, best_match_id, best_match_vector = -1,-1,[]

for user_id, user_vector in enumerate(V.T):
    if my_id != user_id:
        cos_similarity = compute_cos_similarity(my_vector, user_vector)
        if cos_similarity > best_match:
            best_match = cos_similarity
            best_match_id = user_id
            best_match_vector = user_vector

print('best match : {}, best match id : {}'.format(best_match,best_match_id))


best match : 0.9999999951364141, best match id : 1287


In [98]:
recommend_list = []
for i , user_vector in enumerate(adj_matrix):
    if adj_matrix[i][my_id] > 0.9:
        recommend_list.append(i)

print(recommend_list)

[0, 1, 4, 5, 9, 12, 14, 15, 16, 17, 19, 20, 22, 24, 25, 37, 40, 41, 42, 43, 44, 48, 53, 55, 56, 57, 58, 61, 62, 63, 64, 65, 66, 69, 71, 72, 74, 76, 78, 80, 81, 82, 83, 88, 91, 92, 93, 94, 95, 96, 98, 100, 101, 105, 107, 108, 116, 119, 120, 123, 124, 127, 129, 130, 133, 136, 137, 140, 143, 144, 147, 149, 150, 156, 157, 159, 161, 167, 173, 176, 177, 180, 181, 183, 188, 192, 193, 197, 198, 199, 200, 201, 202, 203, 208, 209, 212, 215, 221, 222, 229, 230, 231, 233, 234, 241, 242, 243, 245, 246, 247, 248, 249, 250, 251, 252, 253, 255, 261, 262, 264, 267, 270, 273, 274, 275, 276, 278, 279, 285, 286, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 300, 302, 304, 306, 307, 310, 311, 312, 313, 319, 321, 323, 324, 325, 326, 329, 330, 331, 335, 337, 338, 339, 342, 343, 344, 346, 347, 349, 356, 358, 359, 362, 364, 370, 373, 377, 378, 379, 380, 386, 387, 388, 389, 392, 393, 394, 395, 397, 398, 400, 401, 402, 405, 406, 410, 411, 415, 416, 418, 421, 423, 424, 428, 431, 433, 434, 437, 440, 444, 

* 사용자 기반 추천

* 항목 기반 추천